# Ranking Softwares

Analise voltada a localizar quais softwares mais apontados durante a aberturas de chamado nas subcategorias de `Softwares/Aplicações` dos chamados abertos no GLPI.

Nome do arquivo fonte: `chamados.xlsx`

A base utilizada foi obtida usando o sql abaixo

```sql
select
	*
from
	glpi_tickets a
where
	a.itilcategories_id in (5293, 5294, 5295, 5296, 5297, 5298, 5299) -- ids das categorias sob Software/Aplicacoes
  and a.is_deleted = 0
```


Para obter os Ids das categorias sob `Software/Aplicações` foi utilizado o seguinte SQL:

```sql
SELECT
    a.id `itilcategories_id`,
	a.completename `itilcategorie`,
    CASE WHEN a.level = 5 THEN a.name
		 ELSE NULL END AS 'Categoria.n5',
    CASE WHEN a.level = 4 THEN a.name
		 WHEN a.level = 5 THEN b.name
         ELSE NULL END AS 'Categoria.n4',
    CASE WHEN a.level = 3 THEN a.name
         WHEN a.level = 4 THEN b.name
         WHEN a.level = 5 THEN c.name
         ELSE NULL END AS 'Categoria.n3',
    CASE WHEN a.level = 2 THEN a.name
         WHEN a.level = 3 THEN b.name
         when a.level = 4 then c.name
         when a.level = 5 then d.name
         ELSE NULL END AS 'Categoria.n2',
    CASE WHEN a.level = 1 THEN a.name
         WHEN a.level = 2 THEN b.name
         WHEN a.level = 3 THEN c.name
         WHEN a.level = 4 THEN d.name
         WHEN a.level = 5 THEN e.name
         ELSE NULL END AS 'Categoria.n1'
FROM
    glpi_itilcategories a
	LEFT JOIN glpi_itilcategories b ON a.itilcategories_id = b.id
	LEFT JOIN glpi_itilcategories c ON b.itilcategories_id = c.id
	LEFT JOIN glpi_itilcategories d ON c.itilcategories_id = d.id
	LEFT JOIN glpi_itilcategories e ON d.itilcategories_id = e.id
where
	a.completename like '%Software/Aplicações%'
```

#### Importar Bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#### Funções de Apoio

In [ ]:
def cut_content(content):
    if 'Qual o Software? : </b>' not in content:
        return None
    return content.split('Qual o Software? : </b>')[1].split('</div><div><b>')[0]

def set_quarter_ref(year, quarter):
    return f"{year}-{quarter}"

#### Carregando o DataSet

In [ ]:
df = pd.read_excel('chamados.xlsx')

#### Tratando tabela carregada

In [ ]:
df.date = df.date.apply(pd.to_datetime)
df['software'] = df.content.apply(cut_content)
df['month_ref'] = df.date.dt.strftime('%Y-%m')
df['year_ref'] = df.date.dt.year
df['quarter_ref'] = df.date.dt.quarter
df['quarter_ref'] = df.apply(lambda x: set_quarter_ref(x.year_ref, x.quarter_ref), axis=1)
df.head()

#### Agrupando por software e quadrimestre

In [ ]:
softwares_quarters = df.groupby(by=['software', 'quarter_ref']).size().reset_index(name='count')
quarters_order = softwares_quarters.quarter_ref.unique()
quarters_order.sort()
quarters_order

#### Plotando os graficos para exploracao

In [ ]:
g = sns.FacetGrid(softwares_quarters, col='software', col_wrap=2, height=2, aspect=4)
g.map(sns.pointplot, 'quarter_ref', 'count', order=quarters_order)

#### Validar softwares mais frequentes

Apontando os softwares mais frequentes durante os quadrimestres e plotando para validar o resultado

In [ ]:
## Ranking softwares e instalacao
quarters_count = softwares_quarters.groupby('software').count()

frequent_softwares = quarters_count.query('quarter_ref==8').index.to_list()

In [ ]:
g = sns.FacetGrid(softwares_quarters.query(f'software in {frequent_softwares}'), col='software', col_wrap=2, height=2, aspect=4)
g.map(sns.pointplot, 'quarter_ref', 'count', order=quarters_order)

#### Exportar Resultado

Exportando para o arquivo `ranking_softwares.xlsx.`

In [ ]:
softwares_most_frequent_count = df.query(f'software in {frequent_softwares}').software.value_counts().reset_index(name = 'count')
softwares_most_frequent_count.to_excel('ranking_softwares.xlsx', index=False)